## Лабораторная работа №2

In [2]:
from math import sin, pi

from scipy import signal
import pandas as pd
import cufflinks as cf
import numpy as np

In [3]:
cf.set_config_file(theme='white', offline=True)

### 1. По согласованию с преподавателем выберите одну из систем

In [4]:
# Преобразующая система - px =(1+ap)y, меняется a
a = 1e8

### 2. Вычислите передаточную функцию системы

In [5]:
# W(s) = y(s)/x(s) = s/(1+a*s)

transfer_func = signal.lti([1, 0], [a, 1])

### 3. С помощью scipy.signal вычислите, как меняются полюсы системы при изменении указанного параметра (остальные параметры можно принять равными небольшим целым числам).

In [6]:
# P = -1/a
p_num = 101
vs = np.logspace(1, 7, p_num)
# массив для результатов (так как полюс один, то достаточно массива, а не матрицы
Ps = np.empty(p_num)

for i, vs_i in enumerate(vs):
    _, p, _ = signal.tf2zpk([1, 0], [a / vs_i, 1])
    # полюс всегда гарантированно один, поэтому сохраняем его
    Ps[i] = p[0]
    
df = pd.DataFrame(np.imag(Ps), index=np.real(Ps))
df.iplot(mode='lines + markers', xTitle='Re(P)', yTitle='Im(P)', 
         title=f'a varies from {a / vs[0]} to {a / vs[-1]}')

### 4. Проверьте, какова будет реакция системы, если подать конечный сигнал на неё при параметре, делающем её устойчивой. Затем - при параметре, делающем её неустойчивой (если оказалось, что при всех значениях изменяемого параметра система устойчива либо неустойчива, надо выполнить только одну из этих двух проверок).

In [7]:
# Система всегда устойчива, так как по критерию Гурвица для a*λ+1=0: Единственный главный минор det(1) > 0
# Так что берем любой a
w = 0.001
T = np.linspace(0, 3.0/w, 1001)
S = [sin(t * 2*pi*w) for t in T]
Tout, yout, xout = signal.lsim(transfer_func, S, T)

df_i = pd.DataFrame(S, index=T)
df_i.iplot(mode='lines', xTitle='T', yTitle='S', title='Input')

df_o = pd.DataFrame(yout, index=Tout)
df_o.iplot(mode='lines', xTitle='Tout', yTitle='yout', title='Output')

### 5. Постройте графики АЧХ и ФЧХ для двух систем, описанных в пункте выше. Имеет ли смысл это действие для неустойчивой системы?

In [8]:
W, bode_mag, phase = signal.bode(transfer_func)


df_mwp = pd.DataFrame(bode_mag, index=W)
df_mwp.iplot(mode='lines', xTitle='W', yTitle='mag', title='АЧХ', xaxis_type="log")

df_pwp = pd.DataFrame(phase, index=W)
df_pwp.iplot(mode='lines', xTitle='W', yTitle='phase', title='ФЧХ', xaxis_type="log")

# Имеет ли смысл это действие для неустойчивой системы?
# - Нет. Так как неустойчивая система дает бесконечный рост амплитуды.

### 6. Превратите систему в замкнутую, добавив в неё отрицательную обратную связь выхода со входом. Повторите для полученной системы пункты задания 2-5. Влияет ли обратная связь на устойчивость системы? На АЧХ, ФЧХ?

In [9]:
# Пусть R(s) = s - обратная связь, C(s) = 1, W(s) = s/(1+a*s)
# Тогда Z(s) = C(s)W(s)/(1+C(s)W(s)R(s)) = s/(1+as+s^2)
# Тогда устойчивость зависит от знака a => так как устойчивость зависит от обратной связи => АЧХ и ФЧХ тоже

transfer_func = signal.lti([1, 0], [1, a, 1])

In [10]:
# P = -a/2 ± √(a^2-4)/2
p_num = 101
vs = np.logspace(1, 7, p_num)
# массив для результатов (так как полюс один, то достаточно массива, а не матрицы
Ps = np.empty((2, p_num))

for i, vs_i in enumerate(vs):
    _, p, _ = signal.tf2zpk([1, 0], [1, a / vs_i, 1])
    Ps[0][i] = p[0]
    Ps[1][i] = p[1]
    
df = pd.DataFrame(np.real(Ps).T, columns=['Re(Ps[0])', 'Re(Ps[1])'], index=np.imag(Ps[0]))
df.iplot(mode='lines + markers', xTitle='Im(P)', yTitle='Re(P)',
         title=f'a varies from {a / vs[0]} to {a / vs[-1]}', symbol=['square', 'diamond'])


In [11]:
# Система всегда устойчива, так как по критерию Гурвица для a*λ+1=0: Единственный главный минор a > 0
w = 0.001
T = np.linspace(0, 3.0/w, 1001)
S = [sin(t * 2*pi*w) for t in T]
Tout, yout, xout = signal.lsim(transfer_func, S, T)

df_i = pd.DataFrame(S, index=T)
df_i.iplot(mode='lines', xTitle='T', yTitle='S', title='Input')

df_o = pd.DataFrame(yout, index=Tout)
df_o.iplot(mode='lines', xTitle='Tout', yTitle='yout', title='Output')

In [12]:
W, bode_mag, phase = signal.bode(transfer_func)


df_mwp = pd.DataFrame(bode_mag, index=W)
df_mwp.iplot(mode='lines', xTitle='W', yTitle='mag', title='АЧХ', xaxis_type="log")

df_pwp = pd.DataFrame(phase, index=W)
df_pwp.iplot(mode='lines', xTitle='W', yTitle='phase', title='ФЧХ', xaxis_type="log")